# ECC-ASSIGNMENT-II
DATE - 04/12/24

**Atharva Atul Joshi**

**NY Parking Violations**

In [1]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=da687cccef7ca10ba089023fd57ce95d7921d8a858162a8e014e9f466aba14bb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# Verifying the PySpark installation and initializing its environment

import pyspark
import findspark as fs

# Ensuring that Spark can be found and utilized by the Python environment
fs.init()


In [3]:
# Importing all the required libraries
# Loading necessary libraries for data analysis and ML
import scipy
import numpy as np
import pandas as pd
import sys
import pyspark
from pyspark.sql import Row
from pyspark.sql import functions as spark_funcs
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, when, count, col, sum, regexp_replace, expr, avg, first
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as spark_f
from pyspark.sql.types import IntegerType
from pyspark import SparkContext, SparkConf
from functools import reduce


In [4]:
# Initialize a Spark Session titled 'ECC-Assignment-2' and configure it to utilize 5 CPU cores
spark_session = SparkSession.builder\
        .master("local[5]")\
        .appName("ECC-Assignment-2")\
        .getOrCreate()


In [5]:
# Initialize a Spark Session titled 'ECC-Assignment-2' and configure it to utilize 5 CPU cores


parking_data = spark_session.read.csv("parking01.csv", header=True, inferSchema=True)


In [6]:
parking_data.show()


+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Issu

In [7]:
from pyspark.sql.functions import col, count, when
from functools import reduce

# Define total number of processing segments
processing_segments = 5

# Distribute columns into segments for concurrent analysis
segmented_columns = [parking_data.columns[index::processing_segments] for index in range(processing_segments)]

# Accumulate counts of null values from each segment
aggregate_null_counts = []

for segment_index, columns in enumerate(segmented_columns):
    print(f"Executing segment {segment_index + 1} of {processing_segments}...")

    # Compute the count of nulls for each column within the segment
    segment_null_counts = parking_data.select(
        [count(when(col(column_name).isNull(), column_name)).alias(column_name) for column_name in columns]
    ).collect()

    # Append the computed null counts for this segment
    aggregate_null_counts.append(segment_null_counts)

# Sum the null counts across all segments to get the final tally
final_null_totals = reduce(lambda total, segment: total + segment, aggregate_null_counts)

final_null_totals



Executing segment 1 of 5...
Executing segment 2 of 5...
Executing segment 3 of 5...
Executing segment 4 of 5...
Executing segment 5 of 5...


[Row(Summons Number=0, Violation Code=0, Street Code2=0, Issuer Precinct=0, Time First Observed=416101, Intersecting Street=209502, Days Parking In Effect    =251608, Vehicle Year=0, No Standing or Stopping Violation=476606),
 Row(Plate ID=1, Vehicle Body Type=2596, Street Code3=0, Issuer Code=0, Violation County=6041, Date First Observed=0, From Hours In Effect=317959, Meter Number=413095, Hydrant Violation=476606),
 Row(Registration State=0, Vehicle Make=1362, Vehicle Expiration Date=0, Issuer Command=238332, Violation In Front Of Or Opposite=246368, Law Section=0, To Hours In Effect=317959, Feet From Curb=0, Double Parking Violation=476606),
 Row(Plate Type=0, Issuing Agency=0, Violation Location=238924, Issuer Squad=287986, House Number=249172, Sub Division=169, Vehicle Color=29808, Violation Post Code=322118),
 Row(Issue Date=0, Street Code1=0, Violation Precinct=0, Violation Time=21, Street Name=161, Violation Legal Code=238274, Unregistered Vehicle?=442475, Violation Description

## **PART 1**
# Question - 1
When are tickets most likely to be issued?



In [8]:
# Set up a temporary view for easier SQL operations on the dataset
parking_data.createOrReplaceTempView("parking_view")

# Execute a SQL query to tally occurrences by violation time, focusing on the most frequent
violation_frequency = spark_session.sql('''
    SELECT `Violation Time`, COUNT(*) AS TotalCount
    FROM parking_view
    GROUP BY `Violation Time`
    ORDER BY TotalCount DESC
    LIMIT 30
''')  # Retrieves the top 25 frequent violation times

# Display results: Top 25 frequent violation times

violation_frequency.show(25)


+--------------+----------+
|Violation Time|TotalCount|
+--------------+----------+
|         1140A|       991|
|         1141A|       965|
|         1142A|       944|
|         1145A|       943|
|         1143A|       943|
|         1146A|       914|
|         1139A|       906|
|         0839A|       898|
|         0836A|       893|
|         1150A|       884|
|         1148A|       883|
|         1147A|       876|
|         0840A|       874|
|         1144A|       866|
|         1149A|       865|
|         1153A|       865|
|         1152A|       860|
|         0841A|       858|
|         1136A|       854|
|         0910A|       851|
|         0850A|       849|
|         1155A|       840|
|         0842A|       836|
|         1138A|       832|
|         0845A|       831|
+--------------+----------+
only showing top 25 rows



# Question - 2
What are the most common years and types of cars to be ticketed?





In [9]:
# This SQL query computes the count of violations per vehicle body type and model year,

vehicle_violations_summary = spark_session.sql('''
    SELECT `Vehicle Body Type` AS `Type`, CAST(`Vehicle Year` AS INTEGER) AS `Year`, COUNT(*) AS `Total Violations`
    FROM parking_view
    WHERE `Vehicle Body Type` IS NOT NULL AND `Vehicle Year` > 0
    GROUP BY `Vehicle Body Type`, `Vehicle Year`
    ORDER BY `Total Violations` DESC
''')

# Output the top 15 results to review the most frequent types of vehicle violations
vehicle_violations_summary.show(15)


+----+----+----------------+
|Type|Year|Total Violations|
+----+----+----------------+
|SUBN|2021|           27412|
|SUBN|2022|           24742|
|SUBN|2020|           20657|
|SUBN|2023|           20520|
|SUBN|2019|           19866|
|SUBN|2018|           15989|
|SUBN|2017|           13524|
|SUBN|2016|           10495|
|SUBN|2015|           10133|
|4DSD|2019|            9290|
|4DSD|2017|            8725|
|4DSD|2018|            8609|
|SUBN|2014|            8317|
|4DSD|2020|            8210|
|4DSD|2021|            8206|
+----+----+----------------+
only showing top 15 rows



# Question - 3
Where are tickets most commonly issued?




In [10]:
# This query aggregates the total number of parking violations by location,
# sorting the results to highlight the locations with the highest numbers.

ticket_distribution_by_location = spark_session.sql('''
    SELECT `Violation Location` AS Location, COUNT(*) AS Ticket_Count
    FROM parking_view
    WHERE `Violation Location` IS NOT NULL
    GROUP BY Location
    ORDER BY Ticket_Count DESC
''')

# Display the top 15 violation locations excluding any records where the location is undefined
ticket_distribution_by_location.show(20)

+--------+------------+
|Location|Ticket_Count|
+--------+------------+
|     114|        8887|
|      19|        6243|
|     115|        5509|
|     109|        5455|
|     110|        5146|
|      66|        5111|
|      84|        4991|
|     108|        4787|
|      61|        4758|
|      62|        4730|
|     103|        4638|
|      43|        4564|
|      70|        4518|
|      90|        4501|
|      52|        4475|
|     102|        4358|
|      79|        4337|
|     104|        4304|
|     112|        4282|
|       6|        4043|
+--------+------------+
only showing top 20 rows



In [11]:
# Execute a SQL query to count parking violations at each valid location, explicitly excluding entries with NULL locations.
# The results are sorted to display locations with the most violations.

violation_counts = spark_session.sql('''
    SELECT `Violation Location` AS Location, COUNT(*) AS Tickets
    FROM parking_view
    WHERE `Violation Location` IS NOT NULL
    GROUP BY Location
    ORDER BY Tickets DESC
''')

# Display the top 20 locations with the highest number of tickets
violation_counts.show(20)

+--------+-------+
|Location|Tickets|
+--------+-------+
|     114|   8887|
|      19|   6243|
|     115|   5509|
|     109|   5455|
|     110|   5146|
|      66|   5111|
|      84|   4991|
|     108|   4787|
|      61|   4758|
|      62|   4730|
|     103|   4638|
|      43|   4564|
|      70|   4518|
|      90|   4501|
|      52|   4475|
|     102|   4358|
|      79|   4337|
|     104|   4304|
|     112|   4282|
|       6|   4043|
+--------+-------+
only showing top 20 rows



# Question - 4

Which color of the vehicle is most likely to get a ticket?

In [12]:
# Run a SQL query to tally parking violations per vehicle color, and sort them to identify the most common colors associated with violations.
color_based_violations = spark_session.sql('''
    SELECT `Vehicle Color` AS Color, COUNT(*) AS TotalViolations
    FROM parking_view
    GROUP BY Color
    ORDER BY TotalViolations DESC
''')

# Output the top 25 vehicle colors based on the frequency of parking violations
color_based_violations.show(25)


+-----+---------------+
|Color|TotalViolations|
+-----+---------------+
|   GY|         119580|
|   BK|          99884|
|   WH|          86052|
|   BL|          38675|
| NULL|          29808|
|   RD|          21833|
|WHITE|           9432|
|BLACK|           8850|
|   GR|           6938|
| GREY|           6777|
|  BLK|           5644|
| BLUE|           4083|
|   TN|           4052|
|   BR|           3766|
|  RED|           3410|
|SILVE|           3195|
|  GRY|           2931|
|   GL|           2338|
| GRAY|           2165|
|   MR|           1647|
|  WHT|           1318|
|   OR|           1195|
|   YW|           1174|
|  WHI|           1118|
| LTGY|            761|
+-----+---------------+
only showing top 25 rows



## **PART 2**

KNN

Given a Black vehicle parking illegally at 34510, 10030, 34050 (street codes). What is the probability that it will get an ticket?



In [13]:
import scipy.spatial

# Define a function to identify the closest cluster center to a specific data point.
def closest_cluster_center(point, centers):
    """
    Calculate the nearest cluster center for a given point using squared Euclidean distance.

    Args:
    point: A single data point (list or array).
    centers: An array of cluster centers.

    Returns:
    int: Index of the closest cluster center.
    """
    # Compute distances between the point and each cluster center
    distances = scipy.spatial.distance.cdist([point], centers, metric="sqeuclidean")
    # Determine the index of the smallest distance, identifying the nearest cluster
    closest_center_index = np.argmin(distances)
    return closest_center_index


In [14]:
# Convert street codes to float and select vehicle color for further analysis
processed_parking_data = parking_data.selectExpr(
    "CAST(`Street Code1` AS FLOAT) AS Code1",
    "CAST(`Street Code2` AS FLOAT) AS Code2",
    "CAST(`Street Code3` AS FLOAT) AS Code3",
    "`Vehicle Color` AS Color"
)

# Initialize VectorAssembler to combine street code features into a single feature vector
feature_assembler = VectorAssembler(
    inputCols=["Code1", "Code2", "Code3"],
    outputCol="features"
)

# Apply the VectorAssembler transformation to the dataset to create feature vectors
processed_parking_data = feature_assembler.transform(processed_parking_data)


In [15]:
# Initialize a KMeans model specifying 5 clusters for preliminary analysis
cluster_model = KMeans(k=5)

# Fit the model using the 'features' column of the dataset to find cluster centers
fitted_model = cluster_model.fit(processed_parking_data.select('features'))

# Apply the model to the dataset to assign each data point to a cluster, and cache the results for efficient access
clustered_data = fitted_model.transform(processed_parking_data).cache()





In [16]:
clustered_data.show()


+-------+-------+-------+-----+--------------------+----------+
|  Code1|  Code2|  Code3|Color|            features|prediction|
+-------+-------+-------+-----+--------------------+----------+
|    0.0|    0.0|    0.0| BLUE|           (3,[],[])|         1|
|17870.0|25390.0|32670.0| GRAY|[17870.0,25390.0,...|         3|
|17870.0|25390.0|32670.0| GRAY|[17870.0,25390.0,...|         3|
|12690.0|41700.0|61090.0|WHITE|[12690.0,41700.0,...|         3|
|12690.0|41700.0|61090.0|WHITE|[12690.0,41700.0,...|         3|
|    0.0|    0.0|    0.0|   BK|           (3,[],[])|         1|
| 8690.0|21690.0|21740.0| NULL|[8690.0,21690.0,2...|         3|
|    0.0|61090.0|    0.0|  GRY|   [0.0,61090.0,0.0]|         3|
|    0.0|    0.0|    0.0|  WHT|           (3,[],[])|         1|
|    0.0|40404.0|40404.0| BLUE|[0.0,40404.0,4040...|         3|
|59590.0| 9440.0|    0.0| GREY|[59590.0,9440.0,0.0]|         0|
|37890.0|51290.0|51895.0|WHITE|[37890.0,51290.0,...|         2|
|24090.0|10290.0|52490.0|  BLK|[24090.0,

In [17]:
# Define a list of variations for black as a vehicle color
black_variants = ['BLK', 'BK.', 'BCK', 'BK', 'BLK.', 'Black', 'BC', 'BLAC', 'BK/', 'B LAC']

# Calculate the probability of each cluster having a black vehicle
black_vehicle_prob = clustered_data.groupBy('prediction').agg(
    count(when(col('Color').isin(black_variants), 1)).alias('Black_Count'),
    count('Color').alias('Total_Vehicles')
).orderBy('prediction')

# Calculate the probability of a vehicle being black in each cluster
probability_black = black_vehicle_prob.select(
    'prediction',
    'Black_Count',
    'Total_Vehicles',
    (col('Black_Count') / col('Total_Vehicles')).alias('Probability_Black')
)

# Show the probability of finding a black vehicle in each cluster
probability_black.show()


+----------+-----------+--------------+-------------------+
|prediction|Black_Count|Total_Vehicles|  Probability_Black|
+----------+-----------+--------------+-------------------+
|         0|      11696|         54380| 0.2150790731886723|
|         1|      62968|        252801|0.24908129319108707|
|         2|       7496|         33866|0.22134293982165004|
|         3|      17756|         79865| 0.2223251737306705|
|         4|       5677|         25886|0.21930773391022174|
+----------+-----------+--------------+-------------------+



In [19]:
cluster_centers = np.array(fitted_model.clusterCenters()).astype(float)
street_codes = np.array([34510.0, 10030.0, 34050.0])
cluster_id = closest_cluster_center(street_codes, cluster_centers)

print('Cluster id for Street Code (34510, 10030, 34050) is:', cluster_id)

Cluster id for Street Code (34510, 10030, 34050) is: 0


In [20]:
# Ensure the 'prediction' column and 'cluster_id' are correctly referenced and used
probability_black.filter(col('prediction') == cluster_id).show()


+----------+-----------+--------------+------------------+
|prediction|Black_Count|Total_Vehicles| Probability_Black|
+----------+-----------+--------------+------------------+
|         0|      11696|         54380|0.2150790731886723|
+----------+-----------+--------------+------------------+



**NBA**

Question 1

For each pair of the players (A, B), we define the fear sore of A when facing B is the hit rate, such that B is closet defender when A is shooting. Based on the fear sore, for each player, please find out who is his "most unwanted defender".

In [21]:
# Initialize reading the dataset for NBA shot logs with headers and schema detection enabled
basketball_data = spark_session.read.csv("shot_logs.csv", header=True, inferSchema=True)

# Output the structure of the dataset to verify fields and data types
basketball_data.printSchema()


root
 |-- GAME_ID: integer (nullable = true)
 |-- MATCHUP: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- W: string (nullable = true)
 |-- FINAL_MARGIN: integer (nullable = true)
 |-- SHOT_NUMBER: integer (nullable = true)
 |-- PERIOD: integer (nullable = true)
 |-- GAME_CLOCK: timestamp (nullable = true)
 |-- SHOT_CLOCK: double (nullable = true)
 |-- DRIBBLES: integer (nullable = true)
 |-- TOUCH_TIME: double (nullable = true)
 |-- SHOT_DIST: double (nullable = true)
 |-- PTS_TYPE: integer (nullable = true)
 |-- SHOT_RESULT: string (nullable = true)
 |-- CLOSEST_DEFENDER: string (nullable = true)
 |-- CLOSEST_DEFENDER_PLAYER_ID: integer (nullable = true)
 |-- CLOSE_DEF_DIST: double (nullable = true)
 |-- FGM: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_id: integer (nullable = true)



In [22]:

# Display the initial rows of the NBA shot logs dataset to inspect the data entries
basketball_data.show()


+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
| GAME_ID|             MATCHUP|LOCATION|  W|FINAL_MARGIN|SHOT_NUMBER|PERIOD|         GAME_CLOCK|SHOT_CLOCK|DRIBBLES|TOUCH_TIME|SHOT_DIST|PTS_TYPE|SHOT_RESULT| CLOSEST_DEFENDER|CLOSEST_DEFENDER_PLAYER_ID|CLOSE_DEF_DIST|FGM|PTS|  player_name|player_id|
+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
|21400899|MAR 04, 2015 - CH...|       A|  W|          24|          1|     1|2024-04-13 01:09:00|      10.8|       2|       1.9|      7.7|       2|       made|   Anderson, Alan|                    101187|           1.3|  1|  2|brian roberts|   2031

In [23]:
# Organize the data by player and defender, summarizing shot outcomes
grouped_shot_data = basketball_data.groupBy(
    col("player_id").alias("PlayerID"),
    col("CLOSEST_DEFENDER_PLAYER_ID").alias("DefenderID")
).agg(
    sum(when(col("SHOT_RESULT") == "made", 1).otherwise(0)).alias("MadeShots"),
    sum(when(col("SHOT_RESULT") == "missed", 1).otherwise(0)).alias("MissedShots")
)

# Display the aggregated shot data for review
grouped_shot_data.show()


+--------+----------+---------+-----------+
|PlayerID|DefenderID|MadeShots|MissedShots|
+--------+----------+---------+-----------+
|  203148|    101179|        0|          1|
|  202687|    201980|        1|          0|
|    2744|      1717|        0|          2|
|  203469|    202329|        1|          1|
|  201945|    202322|        0|          3|
|  202689|    202699|        6|          8|
|  202689|    203924|        1|          0|
|  203077|      2730|        1|          0|
|  203077|    201584|        2|          0|
|  202362|    201188|        2|          0|
|  202330|    201978|        2|          1|
|  202324|    203135|        1|          2|
|  203957|      2617|        1|          0|
|    2430|    203092|        3|          2|
|    2430|    200770|        1|          0|
|  202391|    202328|        0|          1|
|  101179|    202390|        0|          1|
|  201961|    203200|        1|          0|
|  202325|    203498|        1|          4|
|  203135|    201951|        1| 

In [24]:
# Creating a temporary view for the NBA DataFrame, replacing it if it already exists
basketball_data.createOrReplaceTempView("nbatemp")

# Executing SQL query to group the data by player and defender, summarizing shot outcomes
grouped_shot_data = spark_session.sql('''
    SELECT player_id AS PlayerID,
           CLOSEST_DEFENDER_PLAYER_ID AS DefenderID,
           SUM(CASE WHEN SHOT_RESULT = 'made' THEN 1 ELSE 0 END) AS MadeShots,
           SUM(CASE WHEN SHOT_RESULT = 'missed' THEN 1 ELSE 0 END) AS MissedShots
    FROM nbatemp
    GROUP BY player_id, CLOSEST_DEFENDER_PLAYER_ID
''')

# Displaying the aggregated shot data
grouped_shot_data.show()


+--------+----------+---------+-----------+
|PlayerID|DefenderID|MadeShots|MissedShots|
+--------+----------+---------+-----------+
|  203148|    101179|        0|          1|
|  202687|    201980|        1|          0|
|    2744|      1717|        0|          2|
|  203469|    202329|        1|          1|
|  201945|    202322|        0|          3|
|  202689|    202699|        6|          8|
|  202689|    203924|        1|          0|
|  203077|      2730|        1|          0|
|  203077|    201584|        2|          0|
|  202362|    201188|        2|          0|
|  202330|    201978|        2|          1|
|  202324|    203135|        1|          2|
|  203957|      2617|        1|          0|
|    2430|    203092|        3|          2|
|    2430|    200770|        1|          0|
|  202391|    202328|        0|          1|
|  101179|    202390|        0|          1|
|  201961|    203200|        1|          0|
|  202325|    203498|        1|          4|
|  203135|    201951|        1| 

In [25]:
# Calculating the hit rate for each combination of player and defender
grouped_shot_data = grouped_shot_data.withColumn("HitRate", expr("MadeShots / (MadeShots + MissedShots)"))

# Dropping duplicate rows and filtering out records with null hit rates
grouped_shot_data = grouped_shot_data.dropDuplicates(["PlayerID", "HitRate"]).filter("HitRate is not null")

# Displaying the modified DataFrame
grouped_shot_data.show()


+--------+----------+---------+-----------+-------------------+
|PlayerID|DefenderID|MadeShots|MissedShots|            HitRate|
+--------+----------+---------+-----------+-------------------+
|  201945|    202322|        0|          3|                0.0|
|  203082|    203490|        0|          1|                0.0|
|  201600|    202324|        1|          3|               0.25|
|  201143|    202700|        7|          2| 0.7777777777777778|
|  202330|    201967|        2|          7| 0.2222222222222222|
|  201600|    201579|        2|          4| 0.3333333333333333|
|  202322|    201565|       13|         16| 0.4482758620689655|
|  101162|    201149|        8|          5| 0.6153846153846154|
|  201939|    201935|       10|          4| 0.7142857142857143|
|     977|    202718|        3|         11|0.21428571428571427|
|  201155|    101145|        1|          4|                0.2|
|  202681|    202689|       11|         17|0.39285714285714285|
|  201228|    101108|        2|         

In [26]:
# Check if the view exists, and if so, drop it before creating a new one
spark_session.sql("DROP VIEW IF EXISTS nba_temp_view")

# Now, safely create a new temporary view for the NBA data
basketball_data.createTempView("nba_temp_view")

# Execute an SQL query to aggregate shot results by player and their closest defender
shot_statistics = spark_session.sql('''
    SELECT
        player_id AS PlayerID,
        CLOSEST_DEFENDER_PLAYER_ID AS DefenderID,
        SUM(CASE WHEN SHOT_RESULT = 'made' THEN 1 ELSE 0 END) AS MadeShots,
        SUM(CASE WHEN SHOT_RESULT = 'missed' THEN 1 ELSE 0 END) AS MissedShots
    FROM nba_temp_view
    GROUP BY player_id, CLOSEST_DEFENDER_PLAYER_ID
''')

# Display the aggregated shot statistics to verify the results
shot_statistics.show()
# Grouping & selecting the min hit rate (each player)
# Grouping & selecting the min hit rate (each player)
nba_final_df = shot_statistics.groupBy("PlayerID").agg({"MadeShots": "min"}).withColumnRenamed("min(MadeShots)", "HitRate")

nba_grouped_df = shot_statistics.join(nba_final_df, ["PlayerID"]).drop("HitRate").select("PlayerID", "DefenderID")

nba_merged_df = nba_grouped_df.join(basketball_data,
                                    (nba_grouped_df["PlayerID"] == basketball_data["player_id"]) &
                                    (nba_grouped_df["DefenderID"] == basketball_data["CLOSEST_DEFENDER_PLAYER_ID"]))



+--------+----------+---------+-----------+
|PlayerID|DefenderID|MadeShots|MissedShots|
+--------+----------+---------+-----------+
|  203148|    101179|        0|          1|
|  202687|    201980|        1|          0|
|    2744|      1717|        0|          2|
|  203469|    202329|        1|          1|
|  201945|    202322|        0|          3|
|  202689|    202699|        6|          8|
|  202689|    203924|        1|          0|
|  203077|      2730|        1|          0|
|  203077|    201584|        2|          0|
|  202362|    201188|        2|          0|
|  202330|    201978|        2|          1|
|  202324|    203135|        1|          2|
|  203957|      2617|        1|          0|
|    2430|    203092|        3|          2|
|    2430|    200770|        1|          0|
|  202391|    202328|        0|          1|
|  101179|    202390|        0|          1|
|  201961|    203200|        1|          0|
|  202325|    203498|        1|          4|
|  203135|    201951|        1| 

In [29]:
# Create a copy of the original DataFrame for checkpointing
nba_df_copy = basketball_data.select("*")


**Question 2**

For each player, we define the comfortable zone of shooting is a matrix of,

{SHOT DIST, CLOSE DEF DIST, SHOT CLOCK}

Please develop a Spark-based algorithm to classify each player's records into 4 comfortable zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul, Stephen Curry, and Lebron James.

In [28]:
# Filtering out rows with null values
basketball_data = basketball_data.na.drop()

# Converting the shot result to binary (1 for 'made', 0 for 'missed')
basketball_data = basketball_data.withColumn("SHOT_RESULT", when(col("SHOT_RESULT") == "made", 1).otherwise(0))


In [31]:
basketball_data.show()


+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
| GAME_ID|             MATCHUP|LOCATION|  W|FINAL_MARGIN|SHOT_NUMBER|PERIOD|         GAME_CLOCK|SHOT_CLOCK|DRIBBLES|TOUCH_TIME|SHOT_DIST|PTS_TYPE|SHOT_RESULT| CLOSEST_DEFENDER|CLOSEST_DEFENDER_PLAYER_ID|CLOSE_DEF_DIST|FGM|PTS|  player_name|player_id|
+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
|21400899|MAR 04, 2015 - CH...|       A|  W|          24|          1|     1|2024-04-13 01:09:00|      10.8|       2|       1.9|      7.7|       2|          1|   Anderson, Alan|                    101187|           1.3|  1|  2|brian roberts|   2031

In [30]:
# List of features to be used for analysis
features_list = ["SHOT_DIST", "CLOSE_DEF_DIST", "SHOT_CLOCK"]

# Convert all specified columns to float data type
for feature in features_list:
    basketball_data = basketball_data.withColumn(feature, basketball_data[feature].cast("float"))

# Display the updated DataFrame
basketball_data.show()


+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
| GAME_ID|             MATCHUP|LOCATION|  W|FINAL_MARGIN|SHOT_NUMBER|PERIOD|         GAME_CLOCK|SHOT_CLOCK|DRIBBLES|TOUCH_TIME|SHOT_DIST|PTS_TYPE|SHOT_RESULT| CLOSEST_DEFENDER|CLOSEST_DEFENDER_PLAYER_ID|CLOSE_DEF_DIST|FGM|PTS|  player_name|player_id|
+--------+--------------------+--------+---+------------+-----------+------+-------------------+----------+--------+----------+---------+--------+-----------+-----------------+--------------------------+--------------+---+---+-------------+---------+
|21400899|MAR 04, 2015 - CH...|       A|  W|          24|          1|     1|2024-04-13 01:09:00|      10.8|       2|       1.9|      7.7|       2|          1|   Anderson, Alan|                    101187|           1.3|  1|  2|brian roberts|   2031

In [33]:
# Vector Assembler to combine features into a single feature vector
vector = VectorAssembler(inputCols=features_list, outputCol="features")

# Transforming the DataFrame using the VectorAssembler and selecting required columns
basketball_data_vec = vector.transform(basketball_data)\
                        .select('player_name', 'features', col('SHOT_RESULT').cast('float').alias('SHOT_RESULT'))

# Displaying the updated DataFrame with vectorized features
basketball_data_vec.show()


+-------------+--------------------+-----------+
|  player_name|            features|SHOT_RESULT|
+-------------+--------------------+-----------+
|brian roberts|[7.69999980926513...|        1.0|
|brian roberts|[28.2000007629394...|        0.0|
|brian roberts|[17.2000007629394...|        0.0|
|brian roberts|[3.70000004768371...|        0.0|
|brian roberts|[18.3999996185302...|        0.0|
|brian roberts|[20.7000007629394...|        0.0|
|brian roberts|[3.5,2.0999999046...|        1.0|
|brian roberts|[24.6000003814697...|        0.0|
|brian roberts|[22.3999996185302...|        0.0|
|brian roberts|[24.5,4.699999809...|        0.0|
|brian roberts|[14.6000003814697...|        1.0|
|brian roberts|[5.90000009536743...|        1.0|
|brian roberts|[26.3999996185302...|        0.0|
|brian roberts|[22.7999992370605...|        0.0|
|brian roberts|[24.7000007629394...|        1.0|
|brian roberts|[25.0,5.400000095...|        0.0|
|brian roberts|[25.6000003814697...|        0.0|
|brian roberts|[24.2

In [34]:
# Initializing KMeans clustering with four clusters
nba_kmeans = KMeans(k=4, seed=1, featuresCol="features")

# Fitting the KMeans model to the vectorized NBA DataFrame
nba_kmeans_model = nba_kmeans.fit(basketball_data_vec)

# List of players mentioned in the question
players_list = ['james harden', 'chris paul', 'stephen curry', 'lebron james']

# Filtering the DataFrame to include only the specified players
filtered_basketball_data_vec = basketball_data_vec.filter(basketball_data_vec['player_name'].isin(players_list))

# Transforming the vectorized NBA DataFrame using the fitted KMeans model
basketball_data_pred = nba_kmeans_model.transform(filtered_basketball_data_vec)\
                        .select('player_name', 'prediction', 'SHOT_RESULT')



In [35]:
# Converting the Spark DataFrame to a Pandas DataFrame for local computations
basketball_data_pred_pd = basketball_data_pred.toPandas()

# Filtering the Pandas DataFrame to include only the specified players
basketball_data_pred_pd = basketball_data_pred_pd[basketball_data_pred_pd['player_name'].isin(players_list)]


In [36]:
# Importing necessary libraries
import pyspark.sql.functions as pysf
from pyspark.sql.window import Window

# Grouping the DataFrame by player_name and prediction, calculating the mean of SHOT_RESULT for each group,
# and sorting the result by player_name and prediction
nba_result = (
    basketball_data_pred
    .groupBy("player_name", "prediction")
    .mean("SHOT_RESULT")
    .sort("player_name", "prediction")
)

# Adding a new column 'maxavg' that contains the maximum average SHOT_RESULT for each player_name
# using a Window function to partition by player_name
best_zone = (
    nba_result
    .withColumn('maxavg', pysf.max('avg(SHOT_RESULT)').over(Window.partitionBy("player_name")))
    .where(pysf.col('avg(SHOT_RESULT)') == pysf.col('maxavg'))
    .drop('maxavg')
)

# Displaying the DataFrame with the best zone for each player
best_zone.show()


+-------------+----------+------------------+
|  player_name|prediction|  avg(SHOT_RESULT)|
+-------------+----------+------------------+
|   chris paul|         3|0.5563380281690141|
| james harden|         1|0.5604395604395604|
| lebron james|         1|0.6613545816733067|
|stephen curry|         1|0.6350710900473934|
+-------------+----------+------------------+



In [37]:
spark_session.stop()
